In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 150
import seaborn as sns
from Bio import SeqIO, Seq

import glob, os, yaml, subprocess, itertools, sparse, vcf
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from statsmodels.distributions.empirical_distribution import ECDF
import sklearn.metrics
from sklearn.decomposition import PCA
import timeit
import scipy.stats as st
import statsmodels.api as sm
import pickle, yaml, tracemalloc
from scipy.stats import binomtest

analysis_dir = '/n/data1/hms/dbmi/farhat/Sanjana/who-mutation-catalogue'
input_data_dir = '/n/data1/hms/dbmi/farhat/ye12/who'
who_variants_combined = pd.read_csv("who_confidence_2021.csv")

import itertools
from stats_utils import *
import warnings
warnings.filterwarnings("ignore")

In [5]:
model_path = "/n/data1/hms/dbmi/farhat/Sanjana/who-mutation-catalogue/Rifampicin/BINARY/tiers=1/phenos=WHO/dropAF_noSyn_unpooled"
model_matrix = pd.read_pickle(os.path.join(model_path, "model_matrix.pkl"))
model_analysis = pd.read_csv(os.path.join(model_path, "model_analysis.csv"))

In [4]:
model_analysis.query("coef==0")

,mutation,coef,pval,BH_pval,Bonferroni_pval,Odds_Ratio,confidence


In [10]:
model_analysis.shape

(820, 7)

In [9]:
model_analysis.query("mutation=='rpoB_p.Asn437Ile'")

,mutation,coef,pval,BH_pval,Bonferroni_pval,Odds_Ratio,confidence


In [11]:
model_matrix.shape

(35362, 843)

In [12]:
825 + 28

853

In [7]:
model_matrix[model_matrix.columns[((model_matrix == 0).all())]]

mutation,rpoB_p.Asn437Ile,rpoB_p.Asn437_Asn438del,rpoB_p.Asp435Leu,rpoB_p.Gln436_Asn437del,rpoB_p.Glu207Gly,rpoB_p.Glu291Ala,rpoB_p.Glu297Asp,rpoB_p.Glu423Gly,rpoB_p.His445Phe,rpoB_p.His445_Lys446del,...,rpoB_p.Pro439Ala,rpoB_p.Pro439Leu,rpoB_p.Ser431_Met434del,rpoB_p.Ser431dup,rpoB_p.Ser450Ala,rpoB_p.Ser450Asn,rpoB_p.Ser450_Leu452del,rpoB_p.Thr427Gly,rpoB_p.Thr427Pro,rpoB_p.Thr427_Ser428del
sample_id,,,,,,,,,,,,,,,,,,,,,
809,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
812,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
815,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
816,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
817,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
933876,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
946364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
946365,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
res = {'fit_time': np.array([0.46307302, 0.46552658, 0.46767092, 0.45148492, 0.46610522]), 
       'score_time': np.array([0.01107359, 0.00999832, 0.01002049, 0.00967932, 0.01317215]), 
       'test_roc_auc': np.array([0.97973397, 0.98943888, 0.95101111, 0.95274882, 0.96620066]), 
       'test_recall': np.array([0.95521739, 0.95523685, 0.89217391, 0.90434783, 0.93217391]), 
       'test_accuracy': np.array([0.97737876, 0.97879259, 0.94937783, 0.95220588, 0.95616516]), 
       'test_balanced_accuracy': np.array([0.97163761, 0.97269387, 0.93456139, 0.93981012, 0.94995116])
      }

res['test_spec'] = 2 * res['test_balanced_accuracy'] - res['test_recall']

In [36]:
res['test_roc_auc'].mean()

0.967826688

In [40]:
res['test_recall'].mean()

0.9278299779999999

In [41]:
res['test_spec'].mean()

0.9796316820000002

In [42]:
res['test_accuracy'].mean()

0.962784044

In [43]:
res['test_balanced_accuracy'].mean()

0.9537308300000001

In [28]:
drug = "Rifampicin"
config_file = "../config_files/binary_07.yaml"
kwargs = yaml.safe_load(open(config_file))

analysis_dir = kwargs["output_dir"]
num_PCs = kwargs["num_PCs"]
tiers_lst = kwargs["tiers_lst"]
pheno_category_lst = kwargs["pheno_category_lst"]

# make sure that both phenotypes are included
if "ALL" in pheno_category_lst:
    phenos_name = "ALL"
    pheno_category_lst = ["ALL", "WHO"]
else:
    phenos_name = "WHO"
    
mutations = pd.read_csv(os.path.join(analysis_dir, drug, "BINARY", "significant_tiers=1+2_variants.csv"))
mutations["Tier"] = mutations["Tier"].astype(str)
mutations = mutations.query("Tier in @tiers_lst").mutation.values

    
############# STEP 1: READ IN THE PREVIOUSLY GENERATED MATRICES #############
    
    
df_phenos = pd.read_csv(os.path.join(analysis_dir, drug, "phenos_binary.csv")).query("phenotypic_category in @pheno_category_lst").set_index("sample_id")
print(f"Fitting model for {len(df_phenos)} {phenos_name} phenotypes on {len(mutations)} Tier {'+'.join(tiers_lst)} mutations!")

# use the model matrix from the unpooled model
matrix = pd.read_pickle(os.path.join(analysis_dir, drug, f"BINARY/tiers={'+'.join(tiers_lst)}/phenos={phenos_name}/dropAF_noSyn_unpooled/model_matrix.pkl"))

if len(set(mutations) - set(matrix.columns)) > 0:
    #raise ValueError(f"Mutations {set(mutations) - set(matrix.columns)} are not in the model matrix")
    mutations = list(set(mutations).intersection(matrix.columns))
    
matrix = matrix[mutations]

# Read in the PC coordinates dataframe, then keep only the desired number of principal components
eigenvec_df = pd.read_csv("../data/eigenvec_10PC.csv", index_col=[0]).iloc[:, :num_PCs]
matrix = matrix.merge(eigenvec_df, left_index=True, right_index=True, how="left")

df_phenos = df_phenos.loc[matrix.index]
assert sum(matrix.index != df_phenos.index.values) == 0

Fitting model for 35401 WHO phenotypes on 231 Tier 1+2 mutations!


In [30]:
matrix

,rpoB_p.His835Arg,rpoB_p.Leu378Arg,rpoB_p.Leu430_Ser431insArg,rpoC_p.Leu507Val,rpoB_p.Ser450Met,Rv2752c_p.Ser189*,rpoC_p.Pro1040Thr,rpoB_p.Gln432Glu,rpoB_p.Leu443Trp,rpoB_p.Asp435Tyr,...,rpoC_p.Glu750Asp,rpoC_p.His525Asn,rpoA_p.Gly31Ala,rpoB_p.Pro700Ser,rpoC_p.Thr812Ile,PC0,PC1,PC2,PC3,PC4
sample_id,,,,,,,,,,,,,,,,,,,,,
809,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000251,0.007680,-0.007175,-0.000713,-0.000181
812,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000240,0.007675,-0.007189,-0.000707,-0.000202
815,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000184,0.007654,-0.007125,-0.000733,-0.000033
816,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000209,0.007694,-0.007159,-0.000706,-0.000184
817,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000235,0.007686,-0.007170,-0.000709,-0.000187
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
933876,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.004837,-0.004983,-0.001063,-0.000114,0.000083
946364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.004849,-0.004961,-0.001047,-0.000157,0.000073
946365,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,-0.004965,-0.002993,0.000277,0.006936,-0.000773


# RIFAMPICIN TEST CASE

# 0. Original Analysis: 137 new significant resistance-associated variants

## But, there are 152 with OR > 1 and BH p-val < 0.01 that are in tier 2 genes (rpoB is the only tier 1)
## These 152 mutations were studied in the next two analyses

In [31]:
folder = "BINARY"
phenos_name = "WHO"
drug = "Rifampicin"
drug_WHO_abbr = "RIF"

# get the number of high confidence resistance-associated 
tiers1_stats = pd.read_csv(os.path.join(analysis_dir, drug, "BINARY/tiers=1/phenos=WHO/dropAF_noSyn_unpooled", "model_analysis_with_stats.csv"))
tiers12_stats = pd.read_csv(os.path.join(analysis_dir, drug, "BINARY/tiers=1+2/phenos=WHO/dropAF_noSyn_unpooled", "model_analysis_with_stats.csv"))

In [32]:
tiers1_stats.query("mutation in ['rpoB_p.Phe503Val', 'rpoB_p.Asn24Lys', 'rpoB_p.Asn501Lys']")

,mutation,predicted_effect,position,confidence,Odds_Ratio,pval,BH_pval,Bonferroni_pval,Significant,Num_Isolates,...,NPV_LB,NPV_UB,Sens_LB,Sens_UB,Spec_LB,Spec_UB,LR+_LB,LR+_UB,LR-_LB,LR-_UB
58,rpoB_p.Asn24Lys,missense_variant,759878,3) Uncertain significance,1.057381,0.0,0.0,0.0,1,3,...,0.669910,0.679703,0.000054,0.000762,0.999845,1.0,NaN,inf,0.999444,1.000034
196,rpoB_p.Phe503Val,missense_variant,761313,3) Uncertain significance,1.019053,0.0,0.0,0.0,1,1,...,0.669872,0.679665,0.000002,0.000484,0.999845,1.0,NaN,inf,0.999743,1.000083
197,rpoB_p.Asn501Lys,missense_variant,761309,3) Uncertain significance,1.019053,0.0,0.0,0.0,1,1,...,0.669872,0.679665,0.000002,0.000484,0.999845,1.0,NaN,inf,0.999743,1.000083


In [3]:
# def get_significant_genes_by_tier(tier1_model, tier2_model, LRT_res):

#     print(f"Total numbers of mutations: {tier1_model.shape}, {tier2_model.shape}")

#     tier1_significant = tier1_model.query("Odds_Ratio > 1 & BH_pval < 0.05")
#     tier2_significant = tier2_model.query("Odds_Ratio > 1 & BH_pval < 0.01")

#     print(f"Numbers of significant mutations: {len(set(tier1_significant.mutation))}, {len(set(tier2_significant.mutation) - set(tier1_significant.mutation))}")
    
#     combined_df =pd.concat([tier1_significant, tier2_significant], axis=0)
#     combined_df["Tier"] = [1]*len(tier1_significant) + [2]*len(tier2_significant)
#     return combined_df.drop_duplicates(subset="mutation", keep="first").merge(LRT_res[["mutation", 'LRT_pval', 'LRT_BH_pval', 'LRT_Bonferroni_pval']], on="mutation", how="left")

# 1. Excluded Samples Analysis: 54 new significant resistance-associated variants

Removed ~5,000 samples that contain both high confidence rpoB mutations (Category 1 or 2 in 2021 catalogue) and any of the ~150 significant tier2 mutations. 

An L2-penalized regression was fit to determine the effect sizes of tier 2 mutations, independent of resistance-associated mutations they may occur with. 

In [4]:
# exclude_df = pd.read_csv(os.path.join(analysis_dir, f"{drug}/{folder}/exclude_comutation/WHOphenos_univariate_stats.csv"))
# exclude_df = exclude_df.loc[~exclude_df["mutation"].str.contains("PC")]
# exclude_df[['Num_Isolates', 'Total_Isolates', 'TP', 'FP', 'TN', 'FN']] = exclude_df[['Num_Isolates', 'Total_Isolates', 'TP', 'FP', 'TN', 'FN']].astype(int)

# exclude_df.query("Odds_Ratio > 1 & BH_pval < 0.01 & TP >= 5 & PPV_LB >= 0.25 & ~mutation.str.contains('rpoB')")

In [5]:
# exclude_df.query("mutation=='rpoC_p.Glu1092Asp'")[['mutation', 'Odds_Ratio', 'OR_LB', 'OR_UB', 'pval', 'BH_pval',
#        'Bonferroni_pval', 'Num_Isolates', 'Total_Isolates', 'TP', 'FP', 'TN',
#        'FN', 'PPV_LB', 'PPV', 'PPV_UB', 'NPV', 'Sens', 'Spec']]

# 2. Likelihood Ratio Test: 5/152 of the Above Tier 2 Mutations are Significant

152 tier 2 variants were found significant in the first analysis. We then performed a likelihood ratio test:

A mutation is removed from the original model input matrix, and then a new model is fit. 

We then compare the log-likelihoods of the original model and the model with 1 less mutation. If the p-value is significant, it means that removing the mutation significantly changes the fit of the model. 

In [53]:
os.path.join(analysis_dir, drug, f"BINARY/tiers=1/phenos={phenos_name}/dropAF_noSyn_unpooled")

'/n/data1/hms/dbmi/farhat/Sanjana/who-mutation-catalogue/Rifampicin/BINARY/tiers=1/phenos=WHO/dropAF_noSyn_unpooled'

In [32]:
phenos_name = "WHO"
out_dir = os.path.join(analysis_dir, drug, f"BINARY/LRT/phenos={phenos_name}")

LRTresults_tier1 = pd.read_csv(os.path.join(out_dir, f"results_tier={'1'}.csv")).rename(columns={"Unnamed: 0": "mutation"})
LRTresults_tier2 = pd.read_csv(os.path.join(out_dir, f"results_tier={'2'}.csv")).rename(columns={"Unnamed: 0": "mutation"})

# remove the FULL model row, which is the first row
LRTresults_tier1 = add_pval_corrections(LRTresults_tier1.iloc[1:, ])
LRTresults_tier2 = add_pval_corrections(LRTresults_tier2.iloc[1:, ])

tier1_model_permute = pd.read_csv(os.path.join(analysis_dir, drug, f"BINARY/tiers=1/phenos={phenos_name}/dropAF_noSyn_unpooled/model_analysis.csv")).query("~mutation.str.contains('PC')")
tier2_model_permute = pd.read_csv(os.path.join(analysis_dir, drug, f"BINARY/tiers=1+2/phenos={phenos_name}/dropAF_noSyn_unpooled/model_analysis.csv")).query("~mutation.str.contains('PC')")

# remove the tier 1 genes for the purposes of this analysis
tier2_model_permute = tier2_model_permute.query("mutation not in @tier1_model_permute.mutation")

# check that all mutations in the permutation dataframe are in the LRT dataframe. The only difference should be the FULL model row
# print(set(tier1_model_permute.mutation).symmetric_difference(LRTresults_tier1.index))
# print(set(tier2_model_permute.mutation).symmetric_difference(LRTresults_tier2.index))
assert len(set(tier1_model_permute.mutation).symmetric_difference(LRTresults_tier1.mutation)) == 0
assert len(set(tier2_model_permute.mutation).symmetric_difference(LRTresults_tier2.mutation)) == 0

# combine results into a single dataframe for easy searching
tier1_model = tier1_model_permute.merge(LRTresults_tier1[["mutation", "pval", "BH_pval", "Bonferroni_pval"]].rename(columns={"pval": "LRT_pval", 
                                                                                                               "BH_pval": "LRT_BH_pval", 
                                                                                                               "Bonferroni_pval": "LRT_Bonferroni_pval"
                                                                                                              }), on="mutation", how="left")

tier2_model = tier2_model_permute.merge(LRTresults_tier2[["mutation", "pval", "BH_pval", "Bonferroni_pval"]].rename(columns={"pval": "LRT_pval", 
                                                                                                               "BH_pval": "LRT_BH_pval", 
                                                                                                               "Bonferroni_pval": "LRT_Bonferroni_pval"
                                                                                                              }), on="mutation", how="left")

print(len(tier1_model), len(tier2_model))

815 2845


In [33]:
# number of Tier 1 mutations determined significant by original analysis, but not in LRT
thresh = 0.05
tier1_model.query("BH_pval < @thresh & LRT_BH_pval > @thresh").confidence.value_counts(dropna=False)

3) Uncertain significance    61
NaN                          19
2) Assoc w R - Interim       15
5) Not assoc w R              5
Name: confidence, dtype: int64

In [34]:
# mutations picked up by the LRT, but not the coefficient association. 
# They are not significant in the tiers 1+2 model either 
thresh = 0.05

print(f"{len(tier1_model.query('Odds_Ratio > 1 & BH_pval < @thresh & LRT_BH_pval > @thresh'))} significant coef, but not LRT")
print(f"{len(tier1_model.query('Odds_Ratio > 1 & BH_pval > @thresh & LRT_BH_pval < @thresh'))} significant LRT, but not coef")
print(f"{len(tier1_model.query('Odds_Ratio > 1 & BH_pval < @thresh & LRT_BH_pval < @thresh'))} significant in both")

78 significant coef, but not LRT
2 significant LRT, but not coef
51 significant in both


In [56]:
thresh = 0.05
len(tier1_model.query('Odds_Ratio > 1 & (BH_pval < @thresh & LRT_BH_pval < @thresh)'))

51

In [58]:
thresh = 0.01
len(tier2_model.query('Odds_Ratio > 1 & (BH_pval < @thresh & LRT_BH_pval < @thresh)'))

2

In [49]:
tier2_model.query('Odds_Ratio > 1 & (BH_pval < @thresh & LRT_BH_pval < @thresh)')

,mutation,coef,pval,BH_pval,Bonferroni_pval,Odds_Ratio,confidence,LRT_pval,LRT_BH_pval,LRT_Bonferroni_pval
2,rpoC_p.Glu1092Asp,0.195840,0.0,0.0,0.0,1.216332,5) Not assoc w R,5.033098e-07,0.000716,0.001432
19,Rv2752c_p.Asn30Ser,0.099958,0.0,0.0,0.0,1.105124,3) Uncertain significance,1.480226e-07,0.000421,0.000421


In [ ]:
# this was the other mutation they drew attention to
tier2_model.query("mutation=='glpK_p.Val192fs'")

,mutation,coef,pval,BH_pval,Bonferroni_pval,Odds_Ratio,confidence,LRT_pval,LRT_BH_pval,LRT_Bonferroni_pval
203,glpK_p.Val192fs,0.026607,0.033,0.262114,1.0,1.026964,NaN,1.0,1.0,1.0


In [50]:
thresh = 0.01

print(f"{len(tier2_model.query('Odds_Ratio > 1 & BH_pval < @thresh & LRT_BH_pval > @thresh'))} significant coef, but not LRT")
print(f"{len(tier2_model.query('Odds_Ratio > 1 & BH_pval > @thresh & LRT_BH_pval < @thresh'))} significant LRT, but not coef")
print(f"{len(tier2_model.query('Odds_Ratio > 1 & BH_pval < @thresh & LRT_BH_pval < @thresh'))} significant in both")

98 significant coef, but not LRT
0 significant LRT, but not coef
2 significant in both


In [51]:
tiers12_stats.query("mutation=='rpoC_p.Glu1092Asp'")[['mutation', 'predicted_effect', 'confidence', 'Odds_Ratio',
       'BH_pval', 'Num_Isolates',
       'Total_Isolates', 'TP', 'FP', 'TN', 'FN', 'PPV', 'NPV', 'Sens', 'Spec',
       'LR+', 'LR-', 'PPV_LB', 'PPV_UB']]

,mutation,predicted_effect,confidence,Odds_Ratio,BH_pval,Num_Isolates,Total_Isolates,TP,FP,TN,FN,PPV,NPV,Sens,Spec,LR+,LR-,PPV_LB,PPV_UB
16,rpoC_p.Glu1092Asp,missense_variant,5) Not assoc w R,1.216332,0.0,2697,30984,1711,986,19935,8352,0.634409,0.704741,0.170029,0.95287,3.60768,0.871022,0.615909,0.652616


In [52]:
thresh = 0.01
tier2_model.query('BH_pval < @thresh & LRT_BH_pval < @thresh')["mutation"].values

array(['rpoC_p.Glu1092Asp', 'Rv2752c_p.Asn30Ser'], dtype=object)

In [71]:
thresh = 0.05
tier1_sig_variants = tier1_model.query('Odds_Ratio > 1 & (BH_pval < @thresh | LRT_BH_pval < @thresh)')["mutation"].values

thresh = 0.01
tier2_sig_variants = tier2_model.query('Odds_Ratio > 1 & (BH_pval < @thresh | LRT_BH_pval < @thresh)')["mutation"].values

In [72]:
len(tier1_sig_variants), len(tier2_sig_variants)

(131, 100)

In [73]:
combined_variants = np.concatenate([tier1_sig_variants, tier2_sig_variants])
print(len(np.unique(combined_variants)))
        
combined_variants_df = pd.DataFrame({"mutation": combined_variants, "Tier": ["1"]*len(tier1_sig_variants) + ["2"]*len(tier2_sig_variants)})

combined_variants_df.to_csv(os.path.join(analysis_dir, drug, "BINARY", "significant_tiers=1+2_variants.csv"), index=False)

231


In [74]:
combined_variants_df

,mutation,Tier
0,rpoB_p.Ser450Leu,1
1,rpoB_p.Asp435Val,1
2,rpoB_p.His445Tyr,1
3,rpoB_p.His445Asp,1
4,rpoB_p.Ser450Trp,1
...,...,...
226,glpK_c.-7A>G,2
227,Rv2477c_p.Phe429Leu,2
228,glpK_p.Asp215Gly,2
229,Rv2752c_p.Ser189*,2


In [85]:
os.path.join(analysis_dir, drug, )

'/n/data1/hms/dbmi/farhat/Sanjana/who-mutation-catalogue/Rifampicin/BINARY/significant_tiers=1+2_variants.csv'

In [ ]:
analysis/prediction_model.py config_files/binary_03.yaml Rifampicin

In [77]:
config_file = "../config_files/binary_03.yaml"
kwargs = yaml.safe_load(open(config_file))

significant_mut_file = os.path.join(analysis_dir, drug, "BINARY", "significant_tiers=1+2_variants.csv")

analysis_dir = kwargs["output_dir"]
num_PCs = kwargs["num_PCs"]
tiers_lst = kwargs["tiers_lst"]
pheno_category_lst = kwargs["pheno_category_lst"]

# make sure that both phenotypes are included
if "ALL" in pheno_category_lst:
    phenos_name = "ALL"
    pheno_category_lst = ["ALL", "WHO"]
else:
    phenos_name = "WHO"

    
df_phenos = pd.read_csv(os.path.join(analysis_dir, drug, "phenos_binary.csv")).query("phenotypic_category in @pheno_category_lst").set_index("sample_id")
mutations = pd.read_csv(significant_mut_file)
mutations["Tier"] = mutations["Tier"].astype(str)
mutations = mutations.query("Tier in @tiers_lst")
print(f"Fitting model for {len(df_phenos)} {phenos_name} phenotypes on {len(mutations)} mutations!")

# read in only the genotypes files for the tiers for this model
df_model = pd.concat([pd.read_csv(os.path.join(analysis_dir, drug, f"genos_{num}.csv.gz"), compression="gzip", low_memory=False) for num in tiers_lst])

# then keep only samples of the desired phenotype
df_model = df_model.loc[df_model["sample_id"].isin(df_phenos.index)]

# keep only mutations for the small model
df_model["mutation"] = df_model["resolved_symbol"] + "_" + df_model["variant_category"]
df_model = df_model.query("mutation in @mutations.mutation")

Fitting model for 35401 WHO phenotypes on 131 mutations!


In [78]:
matrix = df_model.pivot(index="sample_id", columns="mutation", values="variant_binary_status")
del df_model
print(matrix.shape)

matrix = matrix.dropna(axis=0)
print(matrix.shape)

(35401, 131)
(35376, 131)


In [79]:
matrix[matrix.columns[~((matrix == 0).all())]].shape

(35376, 131)

In [81]:
# Read in the PC coordinates dataframe, then keep only the desired number of principal components
eigenvec_df = pd.read_csv("../data/eigenvec_10PC.csv", index_col=[0]).iloc[:, :num_PCs]
matrix = matrix.merge(eigenvec_df, left_index=True, right_index=True, how="left")

df_phenos = df_phenos.loc[matrix.index]
assert sum(matrix.index != df_phenos.index.values) == 0

In [82]:
matrix.shape

(35376, 136)

In [83]:
matrix

,rpoB_c.-142C>T,rpoB_c.-165C>T,rpoB_c.-179C>T,rpoB_c.-180A>C,rpoB_c.-209C>G,rpoB_c.-21G>C,rpoB_p.Ala286Val,rpoB_p.Ala2Thr,rpoB_p.Ala584Asp,rpoB_p.Ala584Gly,...,rpoB_p.Val26Ala,rpoB_p.Val496Ala,rpoB_p.Val496Met,rpoB_p.Val534Met,rpoB_p.Val562Ala,PC0,PC1,PC2,PC3,PC4
sample_id,,,,,,,,,,,,,,,,,,,,,
809,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000251,0.007680,-0.007175,-0.000713,-0.000181
812,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000240,0.007675,-0.007189,-0.000707,-0.000202
815,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000184,0.007654,-0.007125,-0.000733,-0.000033
816,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000209,0.007694,-0.007159,-0.000706,-0.000184
817,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000235,0.007686,-0.007170,-0.000709,-0.000187
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
933876,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.004837,-0.004983,-0.001063,-0.000114,0.000083
946364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.004849,-0.004961,-0.001047,-0.000157,0.000073
946365,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,-0.004965,-0.002993,0.000277,0.006936,-0.000773


In [84]:
df_phenos

,phenotypic_category,phenotype
sample_id,,
809,WHO,0
812,WHO,0
815,WHO,0
816,WHO,0
817,WHO,0
...,...,...
933876,WHO,1
946364,WHO,1
946365,WHO,1


In [ ]:

df_model["mutation"] = df_model["resolved_symbol"] + "_" + df_model["variant_category"]
df_model = df_model.query("mutation in @mutations.mutation")

In [244]:
config_file = "config_files/"
kwargs = yaml.safe_load(open(config_file))

analysis_dir = kwargs["output_dir"]
num_PCs = kwargs["num_PCs"]

pheno_category_lst = kwargs["pheno_category_lst"]
# make sure that both phenotypes are included
if "ALL" in pheno_category_lst:
    phenos_name = "ALL"
    pheno_category_lst = ["ALL", "WHO"]
else:
    phenos_name = "WHO"

    
mutations_lst = pd.read_csv(significat_mut_path, sep="\t", header=None)[0].values
print(f"Fitting model for {phenos_name} phenotypes on {len(mutations_lst)} mutations!")
os.path.join(analysis_dir, drug, "BINARY")

'/n/data1/hms/dbmi/farhat/Sanjana/who-mutation-catalogue/Rifampicin/BINARY'

# Perform bootstrapping for all 152 mutations to see how much AUC, Sens, Spec, and Accuracy Change

If the confidence interval lies above 0, it means that removing the feature SIGNIFICANTLY DECREASES THE PREDICTIVE POWER OF THE MODEL

CI > 0 = FEATURE IS IMPORTANT

In [16]:
# # bootstrapped the difference: full model stat - (model - 1) stat
# LRT_bootstrap_StatsDiff = pd.read_csv(os.path.join(analysis_dir, drug, "BINARY/LRT", f"{phenos_name}phenos_bootstrap_StatsDiff.csv.gz"), compression="gzip")
# print(len(LRT_bootstrap_StatsDiff.mutation.unique()))
# LRT_bootstrap_StatsDiff2 = pd.read_csv(os.path.join(analysis_dir, drug, "BINARY/LRT", f"{phenos_name}phenos_bootstrap_StatsDiff_2.csv.gz"), compression="gzip")
# print(len(LRT_bootstrap_StatsDiff2.mutation.unique()))

# LRT_bootstrap_StatsDiff = pd.concat([LRT_bootstrap_StatsDiff, LRT_bootstrap_StatsDiff2])

In [265]:
# for metric in ['AUC']:

#     print(st.spearmanr(LRT_res.query("mutation in @LRT_bootstrap_StatsDiff.mutation.unique()")["BH_pval"], 
#                  pval_df[metric]
#                 ))

SpearmanrResult(correlation=0.9338806280906226, pvalue=2.6137422278744596e-05)


In [59]:
# pval_df = pd.DataFrame(columns=['AUC', 'Sens', 'Spec', 'accuracy'])

# for mut in LRT_bootstrap_StatsDiff.mutation.unique():
    
#     print("\n" + mut)
    
#     for metric in ['AUC']:
    
# #         lower, upper = np.percentile(LRT_bootstrap_StatsDiff.query("mutation==@mut")[metric], q=[2.5, 97.5])
    
# #         if lower > 0 and upper > 0:
# #             print(f"    {metric}: {round(lower, 4)}, {round(upper, 4)}")

#         pval = 1 - np.mean(LRT_bootstrap_StatsDiff.query('mutation==@mut')[metric].values > 0)
        
#         if pval < 0.05:
#             print(f"{metric}: {round(pval, 2)}")
            
#         pval_df.loc[mut, metric] = pval

# Model with Interaction Terms

Chose 2 variants because they were the only Tier 2 genes that were significantly associated in the Ridge regression and were significant in the LRT

In [60]:
thresh = 0.01
tier2_model.query('BH_pval < @thresh & LRT_BH_pval < @thresh')

,mutation,coef,pval,BH_pval,Bonferroni_pval,Odds_Ratio,confidence,LRT_pval,LRT_BH_pval,LRT_Bonferroni_pval
2,rpoC_p.Glu1092Asp,0.195840,0.0,0.0,0.0,1.216332,5) Not assoc w R,5.033098e-07,0.000716,0.001432
19,Rv2752c_p.Asn30Ser,0.099958,0.0,0.0,0.0,1.105124,3) Uncertain significance,1.480226e-07,0.000421,0.000421


In [61]:
thresh = 0.01
interact_mut = tier2_model.query('BH_pval < @thresh & LRT_BH_pval < @thresh')["mutation"].values
df_phenos = pd.read_csv(os.path.join(analysis_dir, drug, "phenos_binary.csv")).set_index("sample_id")
#interact_mut = ['rpoC_p.Glu1092Asp', 'Rv2752c_p.Asn30Ser']

# model_matrix = pd.read_pickle(os.path.join(analysis_dir, drug, "BINARY/tiers=1+2/phenos=WHO/dropAF_noSyn_unpooled/model_matrix.pkl"))
# interact_matrix = model_matrix.copy()

# # get a list of all pairs of mutations to interact
# interactions = list(itertools.product(*[interact_mut, interact_matrix.columns[interact_matrix.columns.str.contains("rpoB")]]))

# # add interactions
# for (mut1, mut2) in interactions:
#     interact_matrix[f"{mut1}+{mut2}"] = interact_matrix[mut1] * interact_matrix[mut2]
    
# # drop any columns with no signal
# interact_matrix = interact_matrix[interact_matrix.columns[~((interact_matrix == 0).all())]]

# assert len(set(model_matrix.columns) - set(interact_matrix.columns)) == 0
# print(len(interact_matrix), len(model_matrix))

# interact_matrix.to_pickle(os.path.join(analysis_dir, drug, f"BINARY/interaction/model_matrix_{phenos_name}.pkl"))

interact_matrix = pd.read_pickle(os.path.join(analysis_dir, drug, f"BINARY/interaction/model_matrix_{phenos_name}.pkl"))

interact_matrix = interact_matrix.merge(df_phenos, left_index=True, right_index=True, how="left")
del interact_matrix["phenotypic_category"]

interact_matrix.shape

(30984, 3737)

In [62]:
# so much co-occurrence of rpoC_p.Glu1092Asp with rpoB mutations. Can also check proportion that are high confidence rpoB mutations
for mut in interact_mut:
    print(mut, interact_matrix[interact_matrix.columns[interact_matrix.columns.str.contains(mut)]].shape[1])

rpoC_p.Glu1092Asp 125
Rv2752c_p.Asn30Ser 4


In [63]:
phenos_name = "WHO"
interact_res = pd.read_csv(os.path.join(analysis_dir, drug, f"BINARY/interaction/{phenos_name}_coef.csv"))
interact_permute = pd.read_csv(os.path.join(analysis_dir, drug, f"BINARY/interaction/{phenos_name}_coef_permutation.csv"))

# assess significance using the results of the permutation test
for i, row in interact_res.iterrows():
    # p-value is the proportion of permutation coefficients that are AT LEAST AS EXTREME as the test statistic
    if row["coef"] > 0:
        interact_res.loc[i, "pval"] = np.mean(interact_permute[row["mutation"]] >= row["coef"])
    else:
        interact_res.loc[i, "pval"] = np.mean(interact_permute[row["mutation"]] <= row["coef"])
        
interact_res = add_pval_corrections(interact_res)
interact_res["Odds_Ratio"] = np.exp(interact_res["coef"])

interact_res[["Mut1", "Mut2"]] = interact_res["mutation"].str.split("+", expand=True)
interact_res = interact_res.merge(who_variants_combined.query("drug=='RIF'"), left_on="Mut2", right_on="mutation", how="left")
del interact_res["mutation_y"]
del interact_res["drug"]
interact_res.rename(columns={"mutation_x": "mutation"}, inplace=True)

In [68]:
interact_res.query("mutation.str.contains('rpoC_p.Glu1092Asp') & BH_pval < 0.05").sort_values(["BH_pval", "Odds_Ratio"], 
                                                                                           ascending=[True, False]
                                                                                          )

,mutation,coef,pval,BH_pval,Bonferroni_pval,Odds_Ratio,Mut1,Mut2,confidence
3206,rpoC_p.Glu1092Asp,0.219309,0.000,0.000000,0.0,1.245216,rpoC_p.Glu1092Asp,None,NaN
3676,rpoC_p.Glu1092Asp+rpoB_p.Ile491Tyr,0.026609,0.000,0.000000,0.0,1.026966,rpoC_p.Glu1092Asp,rpoB_p.Ile491Tyr,3) Uncertain significance
3693,rpoC_p.Glu1092Asp+rpoB_p.Met434_Asn437delinsIle,0.018817,0.000,0.000000,0.0,1.018995,rpoC_p.Glu1092Asp,rpoB_p.Met434_Asn437delinsIle,2) Assoc w R - Interim
3623,rpoC_p.Glu1092Asp+rpoB_p.Asn260Asp,-0.027974,0.000,0.000000,0.0,0.972414,rpoC_p.Glu1092Asp,rpoB_p.Asn260Asp,NaN
3725,rpoC_p.Glu1092Asp+rpoB_p.Val170Phe,-0.057892,0.003,0.038700,1.0,0.943752,rpoC_p.Glu1092Asp,rpoB_p.Val170Phe,1) Assoc w R
3708,rpoC_p.Glu1092Asp+rpoB_p.Ser450Leu,0.057540,0.004,0.048584,1.0,1.059228,rpoC_p.Glu1092Asp,rpoB_p.Ser450Leu,1) Assoc w R


In [70]:
interact_res.query("mutation.str.contains('rpoB_p.Val170Phe')")

,mutation,coef,pval,BH_pval,Bonferroni_pval,Odds_Ratio,Mut1,Mut2,confidence
2936,rpoB_p.Val170Phe,0.191617,0.000,0.0000,0.0,1.211206,rpoB_p.Val170Phe,None,NaN
3725,rpoC_p.Glu1092Asp+rpoB_p.Val170Phe,-0.057892,0.003,0.0387,1.0,0.943752,rpoC_p.Glu1092Asp,rpoB_p.Val170Phe,1) Assoc w R


In [67]:
interact_res.query("mutation.str.contains('rpoB_p.Asn260Asp')")

,mutation,coef,pval,BH_pval,Bonferroni_pval,Odds_Ratio,Mut1,Mut2,confidence
2395,rpoB_p.Asn260Asp,-0.027974,0.0,0.0,0.0,0.972414,rpoB_p.Asn260Asp,None,NaN
3623,rpoC_p.Glu1092Asp+rpoB_p.Asn260Asp,-0.027974,0.0,0.0,0.0,0.972414,rpoC_p.Glu1092Asp,rpoB_p.Asn260Asp,NaN


In [217]:
interact_matrix.groupby("phenotype")['rpoC_p.Glu1092Asp+rpoB_p.Ser450Leu'].value_counts()

phenotype  rpoC_p.Glu1092Asp+rpoB_p.Ser450Leu
0          0.0                                   20907
           1.0                                      14
1          0.0                                    8682
           1.0                                    1381
Name: rpoC_p.Glu1092Asp+rpoB_p.Ser450Leu, dtype: int64

In [224]:
interact_matrix['rpoC_p.Glu1092Asp+rpoB_p.Ser450Leu'].mean()

0.045023237800154915

In [218]:
1381 / 1385

0.9971119133574007

In [223]:
interact_matrix['rpoC_p.Glu1092Asp'].mean()

0.08704492641363284

In [222]:
interact_matrix['rpoB_p.Ser450Leu'].mean()

0.21449780531887425

In [34]:
os.path.join()

'/n/data1/hms/dbmi/farhat/Sanjana/who-mutation-catalogue/Rifampicin/BINARY/tiers=1/phenos=WHO/dropAF_noSyn_unpooled'